## Adafruit Mini GPS PA1010D GPS  
https://www.adafruit.com/product/4415

In [1]:
import json
import pandas as pd

pd.set_option('max_colwidth', 200)

from meerkat import pa1010d, parser

In [2]:
pd.__version__ #  tested with 1.1.3

'1.1.3'

#### Initialize Driver Class with Default I2C Address

In [3]:
gps = pa1010d.PA1010D(bus_n=1, bus_addr=0x10)

In [4]:
gps.csv_writer.metadata

{'name': 'pa1010d', 'urls': 'https://www.cdtop-tech.com/products/pa1010d', 'manufacturer': 'CDTop Technology', 'header': ['description', 'sample_n', 'nmea_sentence'], 'dtype': ['str', 'int', 'str'], 'precision': '<3.0 meters', 'bus_n': 1, 'bus_addr': '0x10', 'description': 'Adafruit PA1010D GPS/GNSS module', 'supported_nmea_sentences': ['GGA', 'GSA', 'GSV', 'RMC', 'VTG']}

#### NMEA Sentence Support  
The GPS module outputs several NMEA sentences, to collect all the data a continuous looping read is used until enough bytes are collected to reconstruct all the sentences. Like a UART connection, the module cycles through transmitting each supported NMEA sentence type. Therefore, by default all supported sentence types are returned.

The supported NMEA sentences are:  
'GGA', 'GSA', 'GSV', 'RMC', 'VTG'  
Refer to http://aprs.gids.nl/nmea/ and datasheet page 16 for details.

In [5]:
gps.get()

['$GNGGA,042642.000,4741.4569,N,12219.0173,W,1,11,0.86,104.7,M,-17.3,M,,*46',
 '$GPGSA,A,3,15,12,17,14,28,19,,,,,,,1.17,0.86,0.80*05',
 '$GPGSV,4,3,14,02,12,182,26,15,11,240,27,01,08,030,16,13,07,206,22*7E',
 '$GNRMC,042640.000,A,4741.4569,N,12219.0173,W,1.16,320.93,221220,,,A*6E',
 '$GNVTG,336.44,T,,M,1.45,N,2.69,K,A*28']

Using the `nmea_sentences` keyword, specific NMEA sentences can be returned. Note that all sentences are still being transmitted on the I2C bus and being parsed by the driver.

In [6]:
gps.get(nmea_sentences=['GGA', 'GSA'])

['$GNGGA,042645.000,4741.4554,N,12219.0160,W,1,9,0.95,104.7,M,-17.3,M,,*76',
 '$GPGSA,A,3,15,17,14,19,,,,,,,,,1.25,0.95,0.81*0E']

In [7]:
gps.get(nmea_sentences=['GSV', 'RMC', 'VTG'])

['$GLGSV,2,2,08,74,33,317,18,84,28,197,19,65,10,331,,72,07,290,*69',
 '$GNRMC,042647.000,A,4741.4548,N,12219.0154,W,0.14,138.93,221220,,,A*67',
 '$GNVTG,138.93,T,,M,0.14,N,0.25,K,A*21']

In [8]:
gps.get(nmea_sentences=['RMC'])

['$GNRMC,042648.000,A,4741.4547,N,12219.0149,W,0.12,138.93,221220,,,A*6D']

#### CSV Writer Output

In [9]:
gps.metadata.header

['description', 'sample_n', 'nmea_sentence']

In [10]:
gps.write(description="test_1", n=4, nmea_sentences=['GGA', 'GSA'], delay=1)

In [11]:
gps.csv_writer.path

'2020_12_21_20_26_58_pa1010d.csv'

In [12]:
m, df = parser.csv_resource(gps.csv_writer.path)

In [13]:
m

{'encoding': 'utf-8',
 'format': 'text/csv',
 'standard': 'Follow RFC 4180',
 'line_terminator': '\n',
 'quote_char': '"',
 'double_quote': True,
 'escape_char': '\\',
 'null_sequence': 'NA',
 'comment': '#',
 'metadata': {'name': 'pa1010d',
  'urls': 'https://www.cdtop-tech.com/products/pa1010d',
  'manufacturer': 'CDTop Technology',
  'header': ['description', 'sample_n', 'nmea_sentence'],
  'dtype': ['str', 'int', 'str'],
  'precision': '<3.0 meters',
  'bus_n': 1,
  'bus_addr': '0x10',
  'description': 'Adafruit PA1010D GPS/GNSS module',
  'supported_nmea_sentences': ['GGA', 'GSA', 'GSV', 'RMC', 'VTG']},
 'path': '2020_12_21_20_26_58_pa1010d.csv',
 'time_source': 'std_time_ms',
 'time_format': '%Y-%m-%d %H:%M:%S.%f',
 'delimiter': ',',
 'skip_initial_space': True,
 'case_sensitive_header': False,
 'skip_lines': 1}

In [14]:
df.dtypes

std_time_ms              object
description              object
sample_n                  int64
nmea_sentence            object
datetime64_ns    datetime64[ns]
dtype: object

In [15]:
df

,std_time_ms,description,sample_n,nmea_sentence,datetime64_ns
0,2020-12-21 20:26:58.692836,test_1,0,"$GNGGA,042649.000,4741.4549,N,12219.0149,W,1,9,0.92,104.9,M,-17.3,M,,*74",2020-12-21 20:26:58.692836
1,2020-12-21 20:26:58.693026,test_1,0,"$GPGSA,A,3,15,17,14,19,28,,,,,,,,1.25,0.92,0.85*07",2020-12-21 20:26:58.693026
2,2020-12-21 20:26:59.718139,test_1,1,"$GNGGA,042650.000,4741.4549,N,12219.0146,W,1,9,0.92,104.8,M,-17.3,M,,*72",2020-12-21 20:26:59.718139
3,2020-12-21 20:26:59.718488,test_1,1,"$GPGSA,A,3,15,17,14,19,28,,,,,,,,1.25,0.92,0.85*07",2020-12-21 20:26:59.718488
4,2020-12-21 20:27:00.743499,test_1,2,"$GNGGA,042651.000,4741.4548,N,12219.0139,W,1,9,0.92,104.8,M,-17.3,M,,*7A",2020-12-21 20:27:00.743499
5,2020-12-21 20:27:00.743821,test_1,2,"$GPGSA,A,3,15,17,14,19,28,,,,,,,,1.25,0.92,0.85*07",2020-12-21 20:27:00.743821
6,2020-12-21 20:27:01.768962,test_1,3,"$GNGGA,042652.000,4741.4546,N,12219.0136,W,1,9,0.92,104.8,M,-17.3,M,,*78",2020-12-21 20:27:01.768962
7,2020-12-21 20:27:01.769303,test_1,3,"$GPGSA,A,3,15,17,14,19,28,,,,,,,,1.25,0.92,0.85*07",2020-12-21 20:27:01.769303


In [16]:
def nmea_type(sentence):
    """Get NMEA sentence type"""
    s = sentence.split(",")
    return s[0][1:]

In [17]:
df.nmea_sentence.apply(lambda x: nmea_type(sentence=x))

0    GNGGA
1    GPGSA
2    GNGGA
3    GPGSA
4    GNGGA
5    GPGSA
6    GNGGA
7    GPGSA
Name: nmea_sentence, dtype: object

In [18]:
def nmea_parse(sentence):
    """Parse NMEA sentence, removing start of sentence '$' and checksum delimiter '*' """
    start_seq = sentence[0]
    sentence = sentence[1:]
    sentence, checksum = sentence.split("*")
    return sentence.split(",") + [checksum]

In [19]:
df.nmea_sentence.apply(nmea_parse)

0    [GNGGA, 042649.000, 4741.4549, N, 12219.0149, W, 1, 9, 0.92, 104.9, M, -17.3, M, , , 74]
1                       [GPGSA, A, 3, 15, 17, 14, 19, 28, , , , , , , , 1.25, 0.92, 0.85, 07]
2    [GNGGA, 042650.000, 4741.4549, N, 12219.0146, W, 1, 9, 0.92, 104.8, M, -17.3, M, , , 72]
3                       [GPGSA, A, 3, 15, 17, 14, 19, 28, , , , , , , , 1.25, 0.92, 0.85, 07]
4    [GNGGA, 042651.000, 4741.4548, N, 12219.0139, W, 1, 9, 0.92, 104.8, M, -17.3, M, , , 7A]
5                       [GPGSA, A, 3, 15, 17, 14, 19, 28, , , , , , , , 1.25, 0.92, 0.85, 07]
6    [GNGGA, 042652.000, 4741.4546, N, 12219.0136, W, 1, 9, 0.92, 104.8, M, -17.3, M, , , 78]
7                       [GPGSA, A, 3, 15, 17, 14, 19, 28, , , , , , , , 1.25, 0.92, 0.85, 07]
Name: nmea_sentence, dtype: object

In [20]:
df["nmea_type"] = df.nmea_sentence.apply(lambda x: nmea_type(sentence=x))

In [21]:
df

,std_time_ms,description,sample_n,nmea_sentence,datetime64_ns,nmea_type
0,2020-12-21 20:26:58.692836,test_1,0,"$GNGGA,042649.000,4741.4549,N,12219.0149,W,1,9,0.92,104.9,M,-17.3,M,,*74",2020-12-21 20:26:58.692836,GNGGA
1,2020-12-21 20:26:58.693026,test_1,0,"$GPGSA,A,3,15,17,14,19,28,,,,,,,,1.25,0.92,0.85*07",2020-12-21 20:26:58.693026,GPGSA
2,2020-12-21 20:26:59.718139,test_1,1,"$GNGGA,042650.000,4741.4549,N,12219.0146,W,1,9,0.92,104.8,M,-17.3,M,,*72",2020-12-21 20:26:59.718139,GNGGA
3,2020-12-21 20:26:59.718488,test_1,1,"$GPGSA,A,3,15,17,14,19,28,,,,,,,,1.25,0.92,0.85*07",2020-12-21 20:26:59.718488,GPGSA
4,2020-12-21 20:27:00.743499,test_1,2,"$GNGGA,042651.000,4741.4548,N,12219.0139,W,1,9,0.92,104.8,M,-17.3,M,,*7A",2020-12-21 20:27:00.743499,GNGGA
5,2020-12-21 20:27:00.743821,test_1,2,"$GPGSA,A,3,15,17,14,19,28,,,,,,,,1.25,0.92,0.85*07",2020-12-21 20:27:00.743821,GPGSA
6,2020-12-21 20:27:01.768962,test_1,3,"$GNGGA,042652.000,4741.4546,N,12219.0136,W,1,9,0.92,104.8,M,-17.3,M,,*78",2020-12-21 20:27:01.768962,GNGGA
7,2020-12-21 20:27:01.769303,test_1,3,"$GPGSA,A,3,15,17,14,19,28,,,,,,,,1.25,0.92,0.85*07",2020-12-21 20:27:01.769303,GPGSA


#### GGA sentence

Parse using DataFrame and list comprehension for the same result, but loses the row index values

In [22]:
df_gga = pd.DataFrame([nmea_parse(gs) for gs in df.loc[df.nmea_type == "GNGGA", "nmea_sentence"].values])
df_gga.columns = parser.GGA_columns
df_gga

,nmea_type,UTC_time,latitude,NS,longitude,EW,quality,n_satellites,horizontal_dilution,altitude,M,geoidal_separation,M,age_sec_diff,diff_id,checksum
0,GNGGA,042649.000,4741.4549,N,12219.0149,W,1,9,0.92,104.9,M,-17.3,M,,,74
1,GNGGA,042650.000,4741.4549,N,12219.0146,W,1,9,0.92,104.8,M,-17.3,M,,,72
2,GNGGA,042651.000,4741.4548,N,12219.0139,W,1,9,0.92,104.8,M,-17.3,M,,,7A
3,GNGGA,042652.000,4741.4546,N,12219.0136,W,1,9,0.92,104.8,M,-17.3,M,,,78


Parse using .loc and apply to preserve the source indexes

In [23]:
df_gga = df.loc[df.nmea_type == "GNGGA"].apply(lambda x: nmea_parse(x.nmea_sentence), axis=1, result_type='expand')
df_gga.columns = parser.GGA_columns
df_gga

,nmea_type,UTC_time,latitude,NS,longitude,EW,quality,n_satellites,horizontal_dilution,altitude,M,geoidal_separation,M,age_sec_diff,diff_id,checksum
0,GNGGA,042649.000,4741.4549,N,12219.0149,W,1,9,0.92,104.9,M,-17.3,M,,,74
2,GNGGA,042650.000,4741.4549,N,12219.0146,W,1,9,0.92,104.8,M,-17.3,M,,,72
4,GNGGA,042651.000,4741.4548,N,12219.0139,W,1,9,0.92,104.8,M,-17.3,M,,,7A
6,GNGGA,042652.000,4741.4546,N,12219.0136,W,1,9,0.92,104.8,M,-17.3,M,,,78


#### GSA sentence

In [24]:
df_gsa = df.loc[df.nmea_type == "GPGSA"].apply(lambda x: nmea_parse(x.nmea_sentence), axis=1, result_type='expand')
df_gsa.columns = parser.GSA_columns
df_gsa

,nmea_type,mode,mode_fix,sv0,sv1,sv2,sv3,sv4,sv5,sv6,sv7,sv8,sv9,sv10,sv11,pdop,hdop,vdop,checksum
1,GPGSA,A,3,15,17,14,19,28,,,,,,,,1.25,0.92,0.85,07
3,GPGSA,A,3,15,17,14,19,28,,,,,,,,1.25,0.92,0.85,07
5,GPGSA,A,3,15,17,14,19,28,,,,,,,,1.25,0.92,0.85,07
7,GPGSA,A,3,15,17,14,19,28,,,,,,,,1.25,0.92,0.85,07


Merge metadata onto NMEA data

In [25]:
df_meta = df.loc[df.nmea_type == "GNGGA", ["std_time_ms", "description", "sample_n", "datetime64_ns"]]
df_meta

,std_time_ms,description,sample_n,datetime64_ns
0,2020-12-21 20:26:58.692836,test_1,0,2020-12-21 20:26:58.692836
2,2020-12-21 20:26:59.718139,test_1,1,2020-12-21 20:26:59.718139
4,2020-12-21 20:27:00.743499,test_1,2,2020-12-21 20:27:00.743499
6,2020-12-21 20:27:01.768962,test_1,3,2020-12-21 20:27:01.768962


In [26]:
df_gga

,nmea_type,UTC_time,latitude,NS,longitude,EW,quality,n_satellites,horizontal_dilution,altitude,M,geoidal_separation,M,age_sec_diff,diff_id,checksum
0,GNGGA,042649.000,4741.4549,N,12219.0149,W,1,9,0.92,104.9,M,-17.3,M,,,74
2,GNGGA,042650.000,4741.4549,N,12219.0146,W,1,9,0.92,104.8,M,-17.3,M,,,72
4,GNGGA,042651.000,4741.4548,N,12219.0139,W,1,9,0.92,104.8,M,-17.3,M,,,7A
6,GNGGA,042652.000,4741.4546,N,12219.0136,W,1,9,0.92,104.8,M,-17.3,M,,,78


In [27]:
df_final = pd.concat([df_meta, df_gga], axis=1)
df_final

,std_time_ms,description,sample_n,datetime64_ns,nmea_type,UTC_time,latitude,NS,longitude,EW,quality,n_satellites,horizontal_dilution,altitude,M,geoidal_separation,M,age_sec_diff,diff_id,checksum
0,2020-12-21 20:26:58.692836,test_1,0,2020-12-21 20:26:58.692836,GNGGA,042649.000,4741.4549,N,12219.0149,W,1,9,0.92,104.9,M,-17.3,M,,,74
2,2020-12-21 20:26:59.718139,test_1,1,2020-12-21 20:26:59.718139,GNGGA,042650.000,4741.4549,N,12219.0146,W,1,9,0.92,104.8,M,-17.3,M,,,72
4,2020-12-21 20:27:00.743499,test_1,2,2020-12-21 20:27:00.743499,GNGGA,042651.000,4741.4548,N,12219.0139,W,1,9,0.92,104.8,M,-17.3,M,,,7A
6,2020-12-21 20:27:01.768962,test_1,3,2020-12-21 20:27:01.768962,GNGGA,042652.000,4741.4546,N,12219.0136,W,1,9,0.92,104.8,M,-17.3,M,,,78


In [28]:
df_final.dtypes

std_time_ms                    object
description                    object
sample_n                        int64
datetime64_ns          datetime64[ns]
nmea_type                      object
UTC_time                       object
latitude                       object
NS                             object
longitude                      object
EW                             object
quality                        object
n_satellites                   object
horizontal_dilution            object
altitude                       object
M                              object
geoidal_separation             object
M                              object
age_sec_diff                   object
diff_id                        object
checksum                       object
dtype: object

In [29]:
def gps_dd(coord):
    """Convert ddmm.mmmm location to dd.dddd"""
    x = coord.split(".")
    head = x[0]
    minute_decimal = x[1]
    degree = head[0:-2]
    minute = head[-2:]
    return float(degree) + float(minute + "." + minute_decimal) / 60

In [30]:
df_final.latitude.apply(gps_dd)

0    47.690915
2    47.690915
4    47.690913
6    47.690910
Name: latitude, dtype: float64

#### JSON Writer Output

In [31]:
gps.json_writer.metadata_interval = 3
data = gps.publish(description="test_2", n=7, nmea_sentences=['GGA', 'RMC'], delay=2)
for n, d in enumerate(data):
    #print(d)
    #print("-"*40)
    print('line {}'.format(n))
    print('-------')
    print(d)
    print("="*40)
    print()

line 0
-------
{"description": "test_2", "sample_n": 0, "nmea_sentence": "$GNGGA,042653.000,4741.4550,N,12219.0134,W,1,9,0.92,104.8,M,-17.3,M,,*7C", "std_time_ms": "2020-12-21 20:27:03.664038"}

line 1
-------
{"description": "test_2", "sample_n": 0, "nmea_sentence": "$GNRMC,042653.000,A,4741.4550,N,12219.0134,W,0.88,0.53,221220,,,A*6E", "std_time_ms": "2020-12-21 20:27:03.664134"}

line 2
-------
{"description": "test_2", "sample_n": 1, "nmea_sentence": "$GNGGA,042657.000,4741.4569,N,12219.0153,W,1,9,0.92,104.7,M,-17.3,M,,*7C", "std_time_ms": "2020-12-21 20:27:05.760124", "encoding": "utf-8", "format": "text/json", "standard": "RFC 8259", "line_terminator": "\n", "quote_char": "\"", "double_quote": true, "escape_char": "\\", "null_sequence": "NA", "comment": "#", "metadata": {"name": "pa1010d", "urls": "https://www.cdtop-tech.com/products/pa1010d", "manufacturer": "CDTop Technology", "header": ["description", "sample_n", "nmea_sentence"], "dtype": ["str", "int", "str"], "precision": "

In [32]:
# default writer format is CSV, switch to JSON
gps.writer_output = 'json'

In [33]:
# writer method with description and sample number
gps.write(description='test_3', n=15, nmea_sentences=['GGA', 'GSA'])

In [34]:
with open(gps.json_writer.path, 'r') as f:
    for n in range(4):
        #print(f.readline().strip())
        print('line {}'.format(n))
        print('-------')
        print(f.readline())
        print("="*40)
        print()

line 0
-------
{"description": "test_3", "sample_n": 0, "nmea_sentence": "\"$GNGGA,042708.000,4741.4639,N,12219.0244,W,1,8,1.21,106.4,M,-17.3,M,,*7D\"", "std_time_ms": "2020-12-21 20:27:18.407032"}


line 1
-------
{"description": "test_3", "sample_n": 0, "nmea_sentence": "\"$GPGSA,A,3,15,17,19,28,,,,,,,,,1.48,1.21,0.84*01\"", "std_time_ms": "2020-12-21 20:27:18.407392"}


line 2
-------
{"description": "test_3", "sample_n": 1, "nmea_sentence": "\"$GNGGA,042709.000,4741.4644,N,12219.0254,W,1,8,1.21,106.6,M,-17.3,M,,*75\"", "std_time_ms": "2020-12-21 20:27:18.454170", "encoding": "utf-8", "format": "text/json", "standard": "RFC 8259", "line_terminator": "\n", "quote_char": "\"", "double_quote": true, "escape_char": "\\", "null_sequence": "NA", "comment": "#", "metadata": {"name": "pa1010d", "urls": "https://www.cdtop-tech.com/products/pa1010d", "manufacturer": "CDTop Technology", "header": ["description", "sample_n", "nmea_sentence"], "dtype": ["str", "int", "str"], "precision": "<3.0 m

In [35]:
data = []
h = ""
with open(gps.json_writer.path, 'r') as f:
    for _ in range(15):
        s = f.readline().strip()
        js = json.loads(s)
        if "metadata" in js.keys():
            h = [js['time_source']] + js['metadata']['header']
        data.append([js['std_time_ms'] , js['description'], js['sample_n'], js['nmea_sentence']])

In [36]:
h

['std_time_ms', 'description', 'sample_n', 'nmea_sentence']

In [37]:
pd.DataFrame(data, columns=h)

,std_time_ms,description,sample_n,nmea_sentence
0,2020-12-21 20:27:18.407032,test_3,0,"""$GNGGA,042708.000,4741.4639,N,12219.0244,W,1,8,1.21,106.4,M,-17.3,M,,*7D"""
1,2020-12-21 20:27:18.407392,test_3,0,"""$GPGSA,A,3,15,17,19,28,,,,,,,,,1.48,1.21,0.84*01"""
2,2020-12-21 20:27:18.454170,test_3,1,"""$GNGGA,042709.000,4741.4644,N,12219.0254,W,1,8,1.21,106.6,M,-17.3,M,,*75"""
3,2020-12-21 20:27:18.454516,test_3,1,"""$GPGSA,A,3,15,17,19,28,,,,,,,,,1.48,1.21,0.84*01"""
4,2020-12-21 20:27:18.547746,test_3,2,"""$GNGGA,042711.000,4741.4647,N,12219.0264,W,1,8,1.21,107.0,M,-17.3,M,,*7B"""
5,2020-12-21 20:27:18.547941,test_3,2,"""$GLGSA,A,3,82,80,73,84,,,,,,,,,1.48,1.21,0.84*17"""
6,2020-12-21 20:27:18.734311,test_3,3,"""$GNGGA,042713.000,4741.4651,N,12219.0269,W,1,8,1.16,107.2,M,-17.3,M,,*75"""
7,2020-12-21 20:27:18.734524,test_3,3,"""$GPGSA,A,3,15,17,14,19,28,,,,,,,,1.44,1.16,0.85*0D"""
8,2020-12-21 20:27:18.874270,test_3,4,"""$GNGGA,042715.000,4741.4657,N,12219.0280,W,1,8,1.16,107.6,M,-17.3,M,,*76"""
9,2020-12-21 20:27:18.874586,test_3,4,"""$GLGSA,A,3,82,80,84,,,,,,,,,,1.44,1.16,0.85*1A"""
